# In Context Learning - Dataset Preprocess

## Import Libraries

In [1]:
import os
import sys

import pandas as pd
import numpy as np
from src.utils import missing_values    
import warnings
from datetime import datetime

from datasets import load_dataset
import polars as pl

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('max_colwidth', None)

warnings.filterwarnings('ignore')

#### Set Path and Constant Values

In [3]:
# Set Constants

ROOT_PATH = os.getcwd()
CACHE_PATH = os.path.join(ROOT_PATH, 'cache')
DATA_PATH = os.path.join(ROOT_PATH, 'data')
RAW_DATA_PATH = os.path.join(DATA_PATH, 'raw')
SAMPLE_DATA_PATH = os.path.join(DATA_PATH, 'sample')
OUTPUT_DATA_PATH = os.path.join(DATA_PATH, 'output')

# Create Folders
os.makedirs(CACHE_PATH, exist_ok=True)
os.makedirs(RAW_DATA_PATH, exist_ok=True)
os.makedirs(SAMPLE_DATA_PATH, exist_ok=True)
os.makedirs(OUTPUT_DATA_PATH, exist_ok=True)

for sub_folder in ["arc", "hellaswag", "gsm8k"]:
    os.makedirs(os.path.join(SAMPLE_DATA_PATH, sub_folder), exist_ok=True)
    os.makedirs(os.path.join(OUTPUT_DATA_PATH, sub_folder), exist_ok=True)

## Load Dataset: [ARC, HellaSwag, GSM8K]

#### Datasets List

In [4]:
dataset_dict = {
    "arc": {"malhajar/arc-tr" : ["ARC-Challenge", "ARC-Easy"]},
    "hellaswag": {"malhajar/hellaswag-tr": ["train", "validation", "test"]},
    "gsm": {"malhajar/gsm8k_tr-v0.2" : ["test"]},
}

# dataset_dict

#### ARC (AI2 Reasoning Challenge) Dataset

**Amaç:** Muhakeme Yeteneği  
**Detay:** ARC, soyut muhakemeyi test etmek için ilkokul fen bilimleri sorularını değerlendirir ve geniş genel bilgi ve derin muhakeme yetenekleri gerektirir.  
**Örnek Soru:** LLM'lere bir fen bilimleri sorusu sunulur ve muhakeme becerilerini göstererek dört seçenekten doğru cevabı seçmeleri gerekir.    
**Hugginface:** "malhajar/arc-tr" Dataset: https://huggingface.co/datasets/malhajar/arc-tr  

![ARC (AI2 Reasoning Challenge)](img/ARC.png "")

In [6]:
suffix_name = "-v3"
dataset_name = "malhajar/arc-tr" + suffix_name
dataset_name = dataset_name.replace("/", "-") + ".csv.gz"
# dataset_name

splits = {'train': 'ARC-Challenge/train-arc.jsonl', 'test': 'ARC-Challenge/test-arc.jsonl', 'validation': 'ARC-Challenge/validation-arc.jsonl'}
df_data_arc = pd.DataFrame()
for split in ["train", "validation", "test"]:
    df_split = pd.read_json("hf://datasets/malhajar/arc-tr/" + splits[split], lines=True)
    df_data_arc = pd.concat([df_data_arc, df_split], ignore_index=True)

df_data_arc["question"] = df_data_arc.apply(lambda row: f"{row['question']}\nŞıklar:\n" + "\n".join([f"{row['choices']['label'][i]}. {row['choices']['text'][i]}" for i in range(len(row["choices"]["text"]))]), axis=1)
df_data_arc["answer"] = df_data_arc.apply(lambda row: row["choices"]["text"][row["choices"]["label"].index(row["answerKey"])], axis=1)
df_data_arc.to_csv(os.path.join(RAW_DATA_PATH, dataset_name), index=False, encoding='utf-8', compression='gzip')
df_data_arc[["question", "answer", "answerKey"]].head()

,question,answer,answerKey
0,Aşağıdaki ifadelerden hangisi mıknatısların neden genellikle buzdolabı kapısına yapıştığını en iyi şekilde açıklar?\nŞıklar:\nA. Buzdolabının kapısı pürüzsüz.\nB. Buzdolabının kapısı demir içerir.\nC. Buzdolabı kapısı iyi bir iletkendir.\nD. Buzdolabının kapısında elektrik kabloları vardır.,Buzdolabının kapısı demir içerir.,B
1,Sedimanter kaya tabakalarında gözlenen bir kıvrım büyük ihtimalle\nŞıklar:\nA. Akan magmanın soğuması.\nB. Kabuk plakalarının yakınlaşması.\nC. nehir çökeltilerinin birikmesi.\nD. Karbonat minerallerinin çözeltisi.,Kabuk plakalarının yakınlaşması.,B
2,Mezozoik dönemin sonunda birçok bitki ve hayvanın neden öldüğüne dair bilim adamları en son açıklama olarak aşağıdakilerden hangisini sunuyor?\nŞıklar:\nA. dünya çapında hastalık\nB. küresel dağ inşası\nC. bitki ve hayvanları avlayan memelilerin yükselişi\nD. Bir asteroitin çarpması güneş ışığını engelleyen toz yarattı,Bir asteroitin çarpması güneş ışığını engelleyen toz yarattı,D
3,"Bir tekne, kuzeye doğru akan bir nehir akıntısı ve yelkenlerine esen rüzgar tarafından hareket ettirilir. Tekne kuzeydoğuya gidiyor. Rüzgar teknenin yelkenlerine en çok hangi yönde kuvvet uyguluyor?\nŞıklar:\nA. batı\nB. doğu\nC. kuzey\nD. güney",doğu,B
4,Bir buzulun yapıcı kuvvetinin sonucu olan arazi şekli hangisidir?\nŞıklar:\nA. Hareket eden bir buzulun oyduğu vadiler\nB. eriyen buzulun biriktirdiği kaya yığınları\nC. bir buzulun granit yüzeyinde oluşturduğu oyuklar\nD. bir buzulun geçmesiyle sertleşen ana kaya tepeleri,eriyen buzulun biriktirdiği kaya yığınları,B


In [7]:
df_data_arc.shape

(2587, 5)

In [9]:
df_data_arc.head()

,id,question,choices,answerKey,answer
0,MCAS_2009_5_6516,Aşağıdaki ifadelerden hangisi mıknatısların neden genellikle buzdolabı kapısına yapıştığını en iyi şekilde açıklar?\nŞıklar:\nA. Buzdolabının kapısı pürüzsüz.\nB. Buzdolabının kapısı demir içerir.\nC. Buzdolabı kapısı iyi bir iletkendir.\nD. Buzdolabının kapısında elektrik kabloları vardır.,"{'text': ['Buzdolabının kapısı pürüzsüz.', 'Buzdolabının kapısı demir içerir.', 'Buzdolabı kapısı iyi bir iletkendir.', 'Buzdolabının kapısında elektrik kabloları vardır.'], 'label': ['A', 'B', 'C', 'D']}",B,Buzdolabının kapısı demir içerir.
1,Mercury_7233695,Sedimanter kaya tabakalarında gözlenen bir kıvrım büyük ihtimalle\nŞıklar:\nA. Akan magmanın soğuması.\nB. Kabuk plakalarının yakınlaşması.\nC. nehir çökeltilerinin birikmesi.\nD. Karbonat minerallerinin çözeltisi.,"{'text': ['Akan magmanın soğuması.', 'Kabuk plakalarının yakınlaşması.', 'nehir çökeltilerinin birikmesi.', 'Karbonat minerallerinin çözeltisi.'], 'label': ['A', 'B', 'C', 'D']}",B,Kabuk plakalarının yakınlaşması.
2,Mercury_7041615,Mezozoik dönemin sonunda birçok bitki ve hayvanın neden öldüğüne dair bilim adamları en son açıklama olarak aşağıdakilerden hangisini sunuyor?\nŞıklar:\nA. dünya çapında hastalık\nB. küresel dağ inşası\nC. bitki ve hayvanları avlayan memelilerin yükselişi\nD. Bir asteroitin çarpması güneş ışığını engelleyen toz yarattı,"{'text': ['dünya çapında hastalık', 'küresel dağ inşası', 'bitki ve hayvanları avlayan memelilerin yükselişi', 'Bir asteroitin çarpması güneş ışığını engelleyen toz yarattı'], 'label': ['A', 'B', 'C', 'D']}",D,Bir asteroitin çarpması güneş ışığını engelleyen toz yarattı
3,Mercury_7041860,"Bir tekne, kuzeye doğru akan bir nehir akıntısı ve yelkenlerine esen rüzgar tarafından hareket ettirilir. Tekne kuzeydoğuya gidiyor. Rüzgar teknenin yelkenlerine en çok hangi yönde kuvvet uyguluyor?\nŞıklar:\nA. batı\nB. doğu\nC. kuzey\nD. güney","{'text': ['batı', 'doğu', 'kuzey', 'güney'], 'label': ['A', 'B', 'C', 'D']}",B,doğu
4,Mercury_SC_401653,Bir buzulun yapıcı kuvvetinin sonucu olan arazi şekli hangisidir?\nŞıklar:\nA. Hareket eden bir buzulun oyduğu vadiler\nB. eriyen buzulun biriktirdiği kaya yığınları\nC. bir buzulun granit yüzeyinde oluşturduğu oyuklar\nD. bir buzulun geçmesiyle sertleşen ana kaya tepeleri,"{'text': ['Hareket eden bir buzulun oyduğu vadiler', 'eriyen buzulun biriktirdiği kaya yığınları', 'bir buzulun granit yüzeyinde oluşturduğu oyuklar', 'bir buzulun geçmesiyle sertleşen ana kaya tepeleri'], 'label': ['A', 'B', 'C', 'D']}",B,eriyen buzulun biriktirdiği kaya yığınları


#### HellaSwag Dataset

**Amaç:** Sağduyulu çıkarım  
**Detay:** HellaSwag, LLM'lerin sağduyulu çıkarımını, aniden biten bir pasajı tamamlamalarını sağlayarak değerlendirir ve bağlama göre metin devamını anlama ve tahmin etme yeteneklerini test eder.  
**Örnek Soru:** Eksik bir pasaj verilir ve LLM, verilen bağlamdan mantıksal olarak çıkan bir son üretmelidir.  
**Hugginface:** "malhajar/hellaswag-tr" Dataset: https://huggingface.co/datasets/malhajar/hellaswag-tr   

![HellaSwag Dataset](img/HellaSwag.png "")

In [10]:
suffix_name = "-v3"
dataset_name = "malhajar/hellaswag-tr" + suffix_name
dataset_name = dataset_name.replace("/", "-") + ".csv.gz"
# dataset_name

df_data_hellaswag = pd.DataFrame()
splits = {'train': '**/*_train_*.jsonl', 'validation': 'hellaswag_validation_10041.jsonl', 'test': '**/*_test_*.jsonl'}

for split in ["train", "validation"]:
    df_split = pl.read_ndjson('hf://datasets/malhajar/hellaswag-tr/' + splits[split])
    df_split = df_split.to_pandas()
    df_data_hellaswag = pd.concat([df_data_hellaswag, df_split], ignore_index=True)

df_data_hellaswag["question"] = df_data_hellaswag.apply(lambda row: f"{row['ctx']}\nŞıklar:\n" + "\n".join([f"{chr(65+i)}- {option}" for i, option in enumerate(row["endings"])]), axis=1)
df_data_hellaswag["label"] = df_data_hellaswag["label"].astype(int)
df_data_hellaswag["answer"] = df_data_hellaswag.apply(lambda row: row["endings"][row["label"]], axis=1)
df_data_hellaswag["answerKey"] = df_data_hellaswag["label"].apply(lambda x: chr(65 + x))
df_data_hellaswag.to_csv(os.path.join(RAW_DATA_PATH, dataset_name), index=False, encoding='utf-8', compression='gzip')
df_data_hellaswag[["question", "answer", "answerKey"]].head()

,question,answer,answerKey
0,"Beyaz üniformalı bir kadın şef, büyük bir mutfakta bir yığın fırın tepsisini gösteriyor ve bunları sunuyor. tavalar\nŞıklar:\nA- yumurta sarısı ve kabartma tozu içerir.\nB- daha sonra esmer şeker serpilir.\nC- tezgahın üzerindeki süzgeç içerisine yerleştirilir.\nD- hamur işleri ile doldurulur ve fırına yüklenir.",hamur işleri ile doldurulur ve fırına yüklenir.,D
1,"Beyaz üniformalı bir kadın şef, büyük bir mutfakta bir yığın fırın tepsisini gösteriyor ve bunları sunuyor. Tavalar hamur işleri ile doldurulur ve fırına yüklenir. Bir bıçak\nŞıklar:\nA- bir tahta üzerinde hareket ettiği ve içindekileri kestiği görülüyor.\nB- soyulmuş cheesecake, ardından dilimlenmiş muhallebi ve hala pişmiş dondurma geliyor.\nC- pişmiş tavaların iç kısmına bir şekil kazınır.\nD- Silindir şeklindeki hamurların yuvarlak şekilde kesilmesi için kullanılır.",Silindir şeklindeki hamurların yuvarlak şekilde kesilmesi için kullanılır.,D
2,"Fırına bir tepsi patates konulur ve çıkarılır. Büyük bir tepsi kek ters çevrilip tezgaha yerleştirilir. büyük bir tepsi et\nŞıklar:\nA- fırında patatesin üzerine yerleştirilir.\nB- , ls ve turşular fırına konulur.\nC- bir çöplüğe dökülür.\nD- Hazırlandıktan sonra bir yardımcı tarafından fırından çıkarılır.",Hazırlandıktan sonra bir yardımcı tarafından fırından çıkarılır.,D
3,Ortadaki adam mavi gömlek giyen kişinin saç stilini gösteriyor. mavi gömlekli adam\nŞıklar:\nA- mavi gömlek giyen kişinin saçını kesen süngerin üzerinde duruyor.\nB- saç stilini eliyle ve saç spreyiyle yapıyor.\nC- lavabonun yanındaki sandalyeye oturuyor.\nD- göz göze gösteriliyor.,lavabonun yanındaki sandalyeye oturuyor.,C
4,Mavi gömlekli adam lavabonun yanındaki sandalyede oturuyor. Diğer adam saçını yıkamaya başlıyor. O\nŞıklar:\nA- su musluğunu lavaboya düşürür.\nB- sonra lavaboya doğru yürüyor ve ıslak saçlarını sallayarak gülümsüyor.\nC- suyu açar ve sandalyeden kalkar.\nD- şampuanla ovalar ve sonra yıkar.,şampuanla ovalar ve sonra yıkar.,D


In [11]:
df_data_hellaswag['activity_label'].value_counts()

activity_label
Yemek ve Eğlence               4462
Bilgisayarlar ve Elektronik    4169
Sağlık                         3841
Ev ve bahçe                    3203
Kişisel Bakım ve Stil          2627
                               ... 
Rüzgâr sörfü                     48
Malç yaymak                      44
Gitar çalmak                     20
Ana Sayfa,Kategoriler            18
Paspaslama zemini                11
Name: count, Length: 193, dtype: int64

In [12]:
df_data_hellaswag['label'].value_counts()

label
1    12516
0    12500
3    12477
2    12450
Name: count, dtype: int64

#### GSMK8K Dataset

**Amaç:** Matematiksel Muhakeme  
**Detay:** GSM8k, LLM'lerin temel matematik işlemlerini kullanarak çok adımlı matematik problemlerini çözme yeteneklerini test eder. GSM8k, modellerin matematiksel muhakeme ve problem çözme kapasitesini ölçen, çözmek için iki ila sekiz adım gerektiren ilkokul matematik soruları içerir.  
**Örnek Soru:** LLM'nin, doğal dilde açıklanan matematik denklemlerini çözerek soruyu cevaplaması gerekir.  
**Hugginface:** "malhajar/gsm8k_tr-v0.2" Dataset: https://huggingface.co/datasets/malhajar/gsm8k_tr-v0.2

![GSM8K Dataset](img/GSM8K.png "")

In [13]:
suffix_name = "-v2"
dataset_name = "malhajar/gsm8k_tr-v0.2" + suffix_name
dataset_name = dataset_name.replace("/", "-") + ".csv.gz"

for split in ["test"]:
    ds = load_dataset("malhajar/gsm8k_tr-v0.2")
    df_data_gsm = ds[split].to_pandas()
    df_data_gsm.rename(columns={'answer': 'raw_answer'}, inplace=True)

    df_data_gsm['answer'] = df_data_gsm['raw_answer'].str.replace(r"#### \d+", "", regex=True).str.strip()
    df_data_gsm['short_answer'] = df_data_gsm['raw_answer'].apply(lambda x: x.split("#### ")[-1].strip())  # "#### " sonrası kısmı al ve temizle

    df_data_gsm['formated_answer'] = df_data_gsm.apply(lambda row: {"answer": row['answer'],
                                                                "short_answer": row['short_answer']}, axis=1)

df_data_gsm.to_csv(os.path.join(RAW_DATA_PATH, dataset_name), index=False, encoding='utf-8', compression='gzip')
df_data_gsm.head()

,question,raw_answer,answer,short_answer,formated_answer
0,Janet'in ördekleri günde 16 yumurta yapar. Her sabah kahvaltıda üçünü yer ve her gün arkadaşları için dört tanesiyle kek yapar. Geri kalanları her gün çiftçi pazarında taze ördek yumurtası başına 2 dolardan satar. Çiftçi pazarında her gün kaç dolar kazanır?,"Janet her gün 16 yumurta yapar. Her sabah kahvaltısında 3 tanesini yer ve her gün arkadaşları için 4 tanesiyle kek yapar. Bu durumda, 16 - 3 - 4 = 9 adet yumurta satmak için geriye kalır. Çiftçi pazarında her bir taze ördek yumurtasını 2 dolardan sattığına göre, Janet her gün 9 * 2 = 18 dolar kazanır. \n#### 18","Janet her gün 16 yumurta yapar. Her sabah kahvaltısında 3 tanesini yer ve her gün arkadaşları için 4 tanesiyle kek yapar. Bu durumda, 16 - 3 - 4 = 9 adet yumurta satmak için geriye kalır. Çiftçi pazarında her bir taze ördek yumurtasını 2 dolardan sattığına göre, Janet her gün 9 * 2 = 18 dolar kazanır.",18,"{'answer': 'Janet her gün 16 yumurta yapar. Her sabah kahvaltısında 3 tanesini yer ve her gün arkadaşları için 4 tanesiyle kek yapar. Bu durumda, 16 - 3 - 4 = 9 adet yumurta satmak için geriye kalır. Çiftçi pazarında her bir taze ördek yumurtasını 2 dolardan sattığına göre, Janet her gün 9 * 2 = 18 dolar kazanır.', 'short_answer': '18'}"
1,Bir cübbe için 2 top mavi lif ve bunun yarısı kadar beyaz lif gereklidir. Toplamda kaç top lif gereklir?,"Bir cübbe için 2 top mavi lif gerekiyor ve beyaz lif miktarı mavi lifin yarısı kadar olmalıdır. Yani, 2 top mavi lif için 2/2=1 top beyaz lif gerekecektir. Toplamda gereken lif miktarı 2 mavi + 1 beyaz = 3 top lif olacaktır. #### 3","Bir cübbe için 2 top mavi lif gerekiyor ve beyaz lif miktarı mavi lifin yarısı kadar olmalıdır. Yani, 2 top mavi lif için 2/2=1 top beyaz lif gerekecektir. Toplamda gereken lif miktarı 2 mavi + 1 beyaz = 3 top lif olacaktır.",3,"{'answer': 'Bir cübbe için 2 top mavi lif gerekiyor ve beyaz lif miktarı mavi lifin yarısı kadar olmalıdır. Yani, 2 top mavi lif için 2/2=1 top beyaz lif gerekecektir. Toplamda gereken lif miktarı 2 mavi + 1 beyaz = 3 top lif olacaktır.', 'short_answer': '3'}"
2,"Josh bir evi yenileyip satmaya karar verir. 80.000 dolar karşılığında bir ev satın alır ve ardından onarım için 50.000 dolar harcar. Bu, evin değerini %150 artırır. Ne kadar kar etti?","Josh'un ev ve onarım masrafları toplamda 80.000 + 50.000 = $130.000 oldu. Evin değerini %150 artırdı, yani 80.000 * 1,5 = $120.000 değerinde bir artış sağladı. Böylece evin yeni değeri 120.000 + 80.000 = $200.000 oldu. Dolayısıyla Josh 200.000 - 130.000 = $70.000 kar etti. \n#### 70000","Josh'un ev ve onarım masrafları toplamda 80.000 + 50.000 = $130.000 oldu. Evin değerini %150 artırdı, yani 80.000 * 1,5 = $120.000 değerinde bir artış sağladı. Böylece evin yeni değeri 120.000 + 80.000 = $200.000 oldu. Dolayısıyla Josh 200.000 - 130.000 = $70.000 kar etti.",70000,"{'answer': 'Josh'un ev ve onarım masrafları toplamda 80.000 + 50.000 = $130.000 oldu. Evin değerini %150 artırdı, yani 80.000 * 1,5 = $120.000 değerinde bir artış sağladı. Böylece evin yeni değeri 120.000 + 80.000 = $200.000 oldu. Dolayısıyla Josh 200.000 - 130.000 = $70.000 kar etti.', 'short_answer': '70000'}"
3,James haftada 3 kez 3 sprint koşmaya karar verir. Her sprintte 60 metre koşar. Haftada toplam kaç metre koşar?,"James haftada 3 gün, her gün 3 sprint koşmaya karar veriyor. Her bir sprintte 60 metre koştuğuna göre, haftada toplamda 3 gün boyunca 3*3=9 kez sprint yapmış olur. Dolayısıyla, toplamda 9*60=540 metre koşmuş olur. \n#### 540","James haftada 3 gün, her gün 3 sprint koşmaya karar veriyor. Her bir sprintte 60 metre koştuğuna göre, haftada toplamda 3 gün boyunca 3*3=9 kez sprint yapmış olur. Dolayısıyla, toplamda 9*60=540 metre koşmuş olur.",540,"{'answer': 'James haftada 3 gün, her gün 3 sprint koşmaya karar veriyor. Her bir sprintte 60 metre koştuğuna göre, haftada toplamda 3 gün boyunca 3*3=9 kez sprint yapmış olur. Dolayısıyla, toplamda 9*60=540 metre koşmuş olur.', 'short_answer': '540'}"


In [18]:
df_data_gsm.shape

(1317, 5)

In [14]:
# df_data_gsm[df_data_gsm['short_answer'].isna()]

# df_data_gsm['short_answer'].iloc[[392, 487, 1111]]
# df_data_gsm.iloc[[392, 487, 1111]]
# 392, 487, 1111

## Sampling

In [15]:
# DF Lists: 
df_raw_data_list = [df_data_arc, df_data_gsm, df_data_hellaswag]

In [16]:
df_raw_data_list[0].shape

(2587, 5)

In [17]:
raw_dataset_list = [
    {"arc": {"malhajar/arc-tr" : ["ARC-Challenge", "ARC-Easy"]}},
    {"hellaswag": {"malhajar/hellaswag-tr": ["train", "validation", "test"]}},
    {"gsm": {"malhajar/gsm8k_tr-v0.2" : ["test"]}}
]

### GSM8K Dataset

#### 10 Shot Sampling

In [19]:
sample_dataset_name = "gsm_10_sample.csv"
df_data_gsm_10_sample = df_data_gsm.sample(n=10, random_state=34)
df_data_gsm_10_sample.index
df_data_gsm_10_sample.to_csv(os.path.join(SAMPLE_DATA_PATH, "gsm8k", sample_dataset_name), index=False, encoding='utf-8') # , compression='gzip'

Index([124, 1167, 937, 1310, 771, 359, 539, 310, 1006, 68], dtype='int64')

In [20]:
df_data_gsm_10_sample.shape

(10, 5)

In [21]:
df_data_gsm_10_sample

,question,raw_answer,answer,short_answer,formated_answer
124,"Jennie annesinin ofisinde yardım ediyor. Damgalanması gereken 60 mektupluk bir yığını ve zaten damgalanmış mektupların bir yığını var. Damgalanması gereken mektupların üçte birine pul yapıştırıyor. Şimdi damgalanmış mektup yığınında 30 mektup varsa, Jennie başladığında bu yığında kaç mektup vardı?","Jennie 60 mektubun üçte birine, yani 60/3=20 mektuba pul yapıştırıyor. Eğer şu anda damgalanmış mektup yığınında 30 mektup varsa ve Jennie 20 tanesini yeni damgaladıysa, başlangıçta damgalanmış mektup yığınında 30-20=10 mektup vardı. \n\n#### 10","Jennie 60 mektubun üçte birine, yani 60/3=20 mektuba pul yapıştırıyor. Eğer şu anda damgalanmış mektup yığınında 30 mektup varsa ve Jennie 20 tanesini yeni damgaladıysa, başlangıçta damgalanmış mektup yığınında 30-20=10 mektup vardı.",10,"{'answer': 'Jennie 60 mektubun üçte birine, yani 60/3=20 mektuba pul yapıştırıyor. Eğer şu anda damgalanmış mektup yığınında 30 mektup varsa ve Jennie 20 tanesini yeni damgaladıysa, başlangıçta damgalanmış mektup yığınında 30-20=10 mektup vardı.', 'short_answer': '10'}"
1167,"Ronnie'ye 5 dolar verilirken Rissa'ya üç katı kadar verildi. Her biri küçük kız kardeşlerine eşit miktarda para verdikten sonra, Rissa'nın parasının 4/5'i kalmıştır. Ronnie'nin parasında ne kadar kalmıştır?","Rissa'ya 5 doların 3 katı kadar, yani $5 x 3 = $15 verildi. Rissa küçük kız kardeşine bir miktar para verdikten sonra, parasının 4/5'i, yani $15 x 4/5 = $12'si kalmıştır. Bu, Rissa'nın kız kardeşine $15 - $12 = $3 verdiği anlamına gelir. Ronnie de Rissa ile aynı miktarda, yani $3 verdiğinden, Ronnie'nin elinde $5 - $3 = $2 kalmıştır.\n#### 2","Rissa'ya 5 doların 3 katı kadar, yani $5 x 3 = $15 verildi. Rissa küçük kız kardeşine bir miktar para verdikten sonra, parasının 4/5'i, yani $15 x 4/5 = $12'si kalmıştır. Bu, Rissa'nın kız kardeşine $15 - $12 = $3 verdiği anlamına gelir. Ronnie de Rissa ile aynı miktarda, yani $3 verdiğinden, Ronnie'nin elinde $5 - $3 = $2 kalmıştır.",2,"{'answer': 'Rissa'ya 5 doların 3 katı kadar, yani $5 x 3 = $15 verildi. Rissa küçük kız kardeşine bir miktar para verdikten sonra, parasının 4/5'i, yani $15 x 4/5 = $12'si kalmıştır. Bu, Rissa'nın kız kardeşine $15 - $12 = $3 verdiği anlamına gelir. Ronnie de Rissa ile aynı miktarda, yani $3 verdiğinden, Ronnie'nin elinde $5 - $3 = $2 kalmıştır.', 'short_answer': '2'}"
937,"Craig ve kardeşi bildikleri en uzun harfli kelimeleri sırayla heceliyor ve bu kelimelerdeki harf sayısını sayıyorlar. On tur sonunda, Craig her biri 15 harfli 20 kelime heceler. Eğer Craig'in kardeşi, Craig'den toplamda 50 harf fazla hecelenmiş kelimeler hecelerse, on tur sonunda heceledikleri kelimelerdeki toplam harf sayısını hesaplayın.","On tur sonunda, Craig bildiği en uzun harfli kelimelerden her biri 15 harfli olan 20 kelime heceler. Bu durumda Craig toplamda 20*15 = 300 harf hecelemiş olur.\n\nCraig'in kardeşi ise Craig'den toplamda 50 harf fazla hecelemiştir. Bu durumda kardeşi 300+50 = 350 harf hecelemiş olur.\n\nİki kardeşin heceledikleri kelimelerdeki toplam harf sayısı ise 350+300 = 650 harf olur.\n\n#### 650","On tur sonunda, Craig bildiği en uzun harfli kelimelerden her biri 15 harfli olan 20 kelime heceler. Bu durumda Craig toplamda 20*15 = 300 harf hecelemiş olur.\n\nCraig'in kardeşi ise Craig'den toplamda 50 harf fazla hecelemiştir. Bu durumda kardeşi 300+50 = 350 harf hecelemiş olur.\n\nİki kardeşin heceledikleri kelimelerdeki toplam harf sayısı ise 350+300 = 650 harf olur.",650,"{'answer': 'On tur sonunda, Craig bildiği en uzun harfli kelimelerden her biri 15 harfli olan 20 kelime heceler. Bu durumda Craig toplamda 20*15 = 300 harf hecelemiş olur. Craig'in kardeşi ise Craig'den toplamda 50 harf fazla hecelemiştir. Bu durumda kardeşi 300+50 = 350 harf hecelemiş olur. İki kardeşin heceledikleri kelimelerdeki toplam harf sayısı ise 350+300 = 650 harf olur.', 'short_answer': '650'}"
1310,"Tom'un restoranı her gece 6 rezervasyon alıyor. Genellikle her biri 5 d

#### 400 Sampling

In [22]:
sample_dataset_name = "gsm_400_sample.csv"
df_data_gsm_400_sample = df_data_gsm.sample(n=400, random_state=1911)
df_data_gsm_400_sample.index
df_data_gsm_400_sample.to_csv(os.path.join(SAMPLE_DATA_PATH, "gsm8k", sample_dataset_name), index=False, encoding='utf-8') # , compression='gzip'

Index([ 485,  852, 1099,  802,  902,  942,  968,  102,  820,  133,
       ...
        323,  198, 1298, 1092,  310, 1306,  421,  858,  494,  728],
      dtype='int64', length=400)

In [23]:
df_data_gsm_400_sample

,question,raw_answer,answer,short_answer,formated_answer
485,"Amca Ben'in dört atı var. Her at öğün başına beş pound yulaf tüketebilir. Atlarını günde iki kez besler. Bir çuvalda 50 pound yulaf varsa, atlarını beş gün boyunca beslemek için kaç çuval yulafa ihtiyacı vardır?","Amca Ben'in dört atı var ve her bir at öğün başına 5 pound yulaf tüketebiliyor. Bu durumda dört at için bir öğünde toplam 4 x 5 pound = 20 pound yulaf gerekiyor. Atlar günde iki kez beslendiğine göre, günde toplam 2 x 20 pound = 40 pound yulaf tüketiyorlar. Beş gün boyunca atları beslemek için 5 x 40 pound = 200 pound yulafa ihtiyaç var. Bir çuval yulaf 50 pound olduğuna göre, 200 pound yulaf için 200/50 = 4 çuval yulaf gerekiyor. #### 4","Amca Ben'in dört atı var ve her bir at öğün başına 5 pound yulaf tüketebiliyor. Bu durumda dört at için bir öğünde toplam 4 x 5 pound = 20 pound yulaf gerekiyor. Atlar günde iki kez beslendiğine göre, günde toplam 2 x 20 pound = 40 pound yulaf tüketiyorlar. Beş gün boyunca atları beslemek için 5 x 40 pound = 200 pound yulafa ihtiyaç var. Bir çuval yulaf 50 pound olduğuna göre, 200 pound yulaf için 200/50 = 4 çuval yulaf gerekiyor.",4,"{'answer': 'Amca Ben'in dört atı var ve her bir at öğün başına 5 pound yulaf tüketebiliyor. Bu durumda dört at için bir öğünde toplam 4 x 5 pound = 20 pound yulaf gerekiyor. Atlar günde iki kez beslendiğine göre, günde toplam 2 x 20 pound = 40 pound yulaf tüketiyorlar. Beş gün boyunca atları beslemek için 5 x 40 pound = 200 pound yulafa ihtiyaç var. Bir çuval yulaf 50 pound olduğuna göre, 200 pound yulaf için 200/50 = 4 çuval yulaf gerekiyor.', 'short_answer': '4'}"
852,"Bir yerel kasaba genişliyor ve önümüzdeki üç yıl boyunca birkaç yeni ev inşa etmek istiyor. İlk yıl, 12 ev inşa edecekler. Ertesi yıl, bu sayının üç katı kadar ev inşa edecekler. Üçüncü yıl, inşa ettikleri ev sayısını sayacak ve miktarı ikiye katlayacaklar. Kasaba önümüzdeki üç yıl içinde kaç ev inşa etmiş olacak?","İlk yıl, kasaba 12 ev inşa edecek. İkinci yıl, ilk yıla göre üç kat daha fazla ev inşa edecekler, yani 12 * 3 = 36 ev. İlk iki yılda toplamda 36 + 12 = 48 ev inşa edilmiş olacak. Üçüncü yıl, ilk iki yılda inşa edilen ev sayısının iki katı kadar ev inşa edecekler, yani 48 * 2 = 96 ev. Böylece, kasaba önümüzdeki üç yıl içinde toplamda 48 + 96 = 144 ev inşa etmiş olacak. #### 144","İlk yıl, kasaba 12 ev inşa edecek. İkinci yıl, ilk yıla göre üç kat daha fazla ev inşa edecekler, yani 12 * 3 = 36 ev. İlk iki yılda toplamda 36 + 12 = 48 ev inşa edilmiş olacak. Üçüncü yıl, ilk iki yılda inşa edilen ev sayısının iki katı kadar ev inşa edecekler, yani 48 * 2 = 96 ev. Böylece, kasaba önümüzdeki üç yıl içinde toplamda 48 + 96 = 144 ev inşa etmiş olacak.",144,"{'answer': 'İlk yıl, kasaba 12 ev inşa edecek. İkinci yıl, ilk yıla göre üç kat daha fazla ev inşa edecekler, yani 12 * 3 = 36 ev. İlk iki yılda toplamda 36 + 12 = 48 ev inşa edilmiş olacak. Üçüncü yıl, ilk iki yılda inşa edilen ev sayısının iki katı kadar ev inşa edecekler, yani 48 * 2 = 96 ev. Böylece, kasaba önümüzdeki üç yıl içinde toplamda 48 + 96 = 144 ev inşa etmiş olacak.', 'short_answer': '144'}"
1099,"Amber, Micah ve Ahito toplamda 52 mil koştu. Amber 8 mil koştu. Micah, Amber'ın koştuğunun 3.5 katı kadar koştu. Ahito kaç mil koştu?","Amber 8 mil koştu.\nMicah, Amber'ın koştuğunun 3.5 katı kadar koştu yani 3.5 * 8 mil = 28 mil koştu.\nAhito kalan mesafeyi koştu = 52 - 8 - 28 = 16 mil koştu.\nAhito 16 mil koştu.\n#### 16","Amber 8 mil koştu.\nMicah, Amber'ın koştuğunun 3.5 katı kadar koştu yani 3.5 * 8 mil = 28 mil koştu.\nAhito kalan mesafeyi koştu = 52 - 8 - 28 = 16 mil koştu.\nAhito 16 mil koştu.",16,"{'answer': 'Amber 8 mil koştu. Micah, Amber'ın koştuğunun 3.5 katı kadar koştu yani 3.5 * 8 mil = 28 mil koştu. Ahito kalan mesafeyi koştu = 52 - 8 - 28 = 16 mil koştu. Ahito 16 mil koştu.', 'short_answer': '16'}"
802,"İki araba bir otobanda seyahat ediyor. İlk araba saatte ortalama 60 mil hızla giderken, ikinci araba saatte ortalama 70 mil hızla onu geçiyor.

#### Ensemble Method-1: 10 Örnekten Tekrarlı 10’lu Seçimler (Replacement=True)

In [24]:
for i in range(1, 11):
    print(f"Ensemble Method-1 Size:10_{i}")
    ens_1_dataset_name = f"gsm_10_sample_ens_1_{i}.csv"
    df_data_ens_1_sample = df_data_gsm_10_sample.sample(n=10, replace=True, random_state=i+5)
    df_data_ens_1_sample.to_csv(os.path.join(SAMPLE_DATA_PATH, "gsm8k", ens_1_dataset_name), index=False, encoding='utf-8') # , compression='gzip'
    print(df_data_ens_1_sample.index)
    print()

Ensemble Method-1 Size:10_1
Index([68, 1310, 771, 124, 68, 1167, 68, 1167, 771, 1167], dtype='int64')

Ensemble Method-1 Size:10_2
Index([771, 68, 539, 1310, 1310, 310, 310, 68, 310, 1006], dtype='int64')

Ensemble Method-1 Size:10_3
Index([1310, 771, 1167, 68, 359, 1006, 1310, 1006, 124, 359], dtype='int64')

Ensemble Method-1 Size:10_4
Index([359, 539, 1006, 539, 1167, 539, 771, 1006, 1167, 1006], dtype='int64')

Ensemble Method-1 Size:10_5
Index([68, 771, 124, 1167, 68, 124, 1167, 1006, 68, 124], dtype='int64')

Ensemble Method-1 Size:10_6
Index([68, 124, 1167, 310, 1167, 310, 937, 1006, 124, 124], dtype='int64')

Ensemble Method-1 Size:10_7
Index([539, 1167, 937, 1310, 1310, 124, 539, 1167, 771, 359], dtype='int64')

Ensemble Method-1 Size:10_8
Index([937, 124, 124, 539, 937, 771, 68, 1310, 771, 937], dtype='int64')

Ensemble Method-1 Size:10_9
Index([1006, 539, 310, 68, 539, 124, 1006, 68, 310, 539], dtype='int64')

Ensemble Method-1 Size:10_10
Index([1006, 359, 359, 310, 124, 310

#### Ensemble Method-2: 10 Örnekten 5’li Seçimler (Replacement=False)

In [25]:
for i in range(1, 11):
    print(f"Ensemble Method-2 Size:10_{i}")
    ens_2_dataset_name = f"gsm_10_sample_ens_2_{i}.csv"
    df_data_ens_2_sample = df_data_gsm_10_sample.sample(n=5, replace=False, random_state=i+20)
    df_data_ens_2_sample.to_csv(os.path.join(SAMPLE_DATA_PATH, "gsm8k", ens_2_dataset_name), index=False, encoding='utf-8') # , compression='gzip'
    print(df_data_ens_2_sample.index)
    print()

Ensemble Method-2 Size:10_1
Index([1167, 937, 310, 1310, 359], dtype='int64')

Ensemble Method-2 Size:10_2
Index([937, 1167, 68, 539, 1310], dtype='int64')

Ensemble Method-2 Size:10_3
Index([359, 1006, 937, 68, 771], dtype='int64')

Ensemble Method-2 Size:10_4
Index([68, 771, 1006, 310, 359], dtype='int64')

Ensemble Method-2 Size:10_5
Index([359, 1167, 1310, 1006, 124], dtype='int64')

Ensemble Method-2 Size:10_6
Index([1006, 771, 68, 310, 937], dtype='int64')

Ensemble Method-2 Size:10_7
Index([771, 937, 68, 1167, 359], dtype='int64')

Ensemble Method-2 Size:10_8
Index([68, 937, 1006, 310, 1310], dtype='int64')

Ensemble Method-2 Size:10_9
Index([1006, 539, 1167, 310, 124], dtype='int64')

Ensemble Method-2 Size:10_10
Index([124, 1167, 937, 1310, 539], dtype='int64')



#### Ensemble Method-3: 10 Örnekten Tekrarsız 10’lu Seçimler (Replacement=False)

In [26]:
for i in range(1, 11):
    print(f"Ensemble Method-3 Size:10_{i}")
    ens_3_dataset_name = f"gsm_10_sample_ens_3_{i}.csv"
    df_data_ens_3_sample = df_data_gsm_10_sample.sample(n=10, replace=False, random_state=i+35)
    df_data_ens_3_sample.to_csv(os.path.join(SAMPLE_DATA_PATH, "gsm8k", ens_3_dataset_name), index=False, encoding='utf-8') # , compression='gzip'
    print(df_data_ens_3_sample.index)
    print()

Ensemble Method-3 Size:10_1
Index([1310, 68, 310, 771, 1006, 1167, 124, 539, 937, 359], dtype='int64')

Ensemble Method-3 Size:10_2
Index([771, 68, 124, 1167, 310, 1006, 1310, 937, 539, 359], dtype='int64')

Ensemble Method-3 Size:10_3
Index([1006, 124, 310, 68, 937, 539, 771, 1310, 359, 1167], dtype='int64')

Ensemble Method-3 Size:10_4
Index([937, 1167, 310, 1006, 359, 1310, 539, 771, 124, 68], dtype='int64')

Ensemble Method-3 Size:10_5
Index([771, 1310, 1006, 1167, 937, 68, 124, 359, 310, 539], dtype='int64')

Ensemble Method-3 Size:10_6
Index([359, 539, 310, 1006, 1167, 68, 937, 771, 1310, 124], dtype='int64')

Ensemble Method-3 Size:10_7
Index([1006, 1167, 359, 124, 310, 937, 68, 771, 1310, 539], dtype='int64')

Ensemble Method-3 Size:10_8
Index([1310, 68, 539, 1006, 937, 359, 1167, 310, 124, 771], dtype='int64')

Ensemble Method-3 Size:10_9
Index([937, 310, 68, 124, 539, 1006, 359, 1167, 1310, 771], dtype='int64')

Ensemble Method-3 Size:10_10
Index([937, 771, 1167, 1006, 68, 53

### Arc Dataset

#### 10 Shot Sampling

In [27]:
sample_dataset_name = "arc_10_sample.csv"
df_data_arc_10_sample = df_data_arc.sample(n=10, random_state=34)
df_data_arc_10_sample.index
df_data_arc_10_sample.to_csv(os.path.join(SAMPLE_DATA_PATH, "arc", sample_dataset_name), index=False, encoding='utf-8')

Index([130, 1312, 447, 1476, 1063, 743, 610, 690, 2554, 580], dtype='int64')

In [28]:
df_data_arc_10_sample.shape

(10, 5)

In [34]:
df_data_arc_10_sample

,id,question,choices,answerKey,answer
130,ACTAAP_2008_7_3,Bilimsel bir araştırmayla hangi soru büyük olasılıkla belirlenebilir?\nŞıklar:\nA. Bir sonraki lotoyu kim kazanacak?\nB. Bir sonraki maçı hangi futbol takımı kazanacak?\nC. Domates yetiştirmek için gereken ışık miktarı nedir?\nD. Hangi dört kuş tüyü türü en güzel renklere sahiptir?,"{'text': ['Bir sonraki lotoyu kim kazanacak?', 'Bir sonraki maçı hangi futbol takımı kazanacak?', 'Domates yetiştirmek için gereken ışık miktarı nedir?', 'Hangi dört kuş tüyü türü en güzel renklere sahiptir?'], 'label': ['A', 'B', 'C', 'D']}",C,Domates yetiştirmek için gereken ışık miktarı nedir?
1312,Mercury_7115273,Bir gölün yüzeyindeki büyük alg çoğalmaları hangi abiyotik faktörün dibe ulaşmasını engeller?\nŞıklar:\nA. su\nB. tuz\nC. ışık\nD. oksijen,"{'text': ['su', 'tuz', 'ışık', 'oksijen'], 'label': ['A', 'B', 'C', 'D']}",C,ışık
447,MCAS_2008_8_5608,"Yeni bir organizma keşfedilirse, onu uygun krallığa sınıflandırmak için aşağıdakilerden hangisi büyük olasılıkla kullanılır?\nŞıklar:\nA. organizmanın rengi\nB. organizmanın doğal yaşam alanı\nC. organizmanın anatomisinin yapısı\nD. organizmanın bulunduğu yer","{'text': ['organizmanın rengi', 'organizmanın doğal yaşam alanı', 'organizmanın anatomisinin yapısı', 'organizmanın bulunduğu yer'], 'label': ['A', 'B', 'C', 'D']}",C,organizmanın anatomisinin yapısı
1476,Mercury_SC_416529,Neden orman ağaçlarının yapraklarının çoğu ağacın tepesinde yetişiyor?\nŞıklar:\nA. tohumları dağıtmak\nB. güneş ışığını yakalamak için\nC. bagajı gölgelemek\nD. su toplamak,"{'text': ['tohumları dağıtmak', 'güneş ışığını yakalamak için', 'bagajı gölgelemek', 'su toplamak'], 'label': ['A', 'B', 'C', 'D']}",B,güneş ışığını yakalamak için
1063,Mercury_183978,Bunlardan hangisi yırtıcı hayvanlara karşı savunmaya yönelik bir adaptasyon örneğidir?\nŞıklar:\nA. zürafanın uzun boynu\nB. yavru geyik lekeleri\nC. su samuru yağlı kürkü\nD. balığın solungaçları,"{'text': ['zürafanın uzun boynu', 'yavru geyik lekeleri', 'su samuru yağlı kürkü', 'balığın solungaçları'], 'label': ['A', 'B', 'C', 'D']}",B,yavru geyik lekeleri
743,LEAP_2008_4_10288,"Bir bölgede çok fazla balık avlanması ekosistemi nasıl etkiler?\nŞıklar:\nA. Balıklar, yakalanan balıkların yerine çok daha fazla yumurta bırakacaktır.\nB. Balıkları yiyen organizmalar açlık nedeniyle tehlike altına girebilir.\nC. Balıkların yediği organizmalar tehlike altına girecek.\nD. İnsanlar çok fazla balık yiyip hastalanabilirler.","{'text': ['Balıklar, yakalanan balıkların yerine çok daha fazla yumurta bırakacaktır.', 'Balıkları yiyen organizmalar açlık nedeniyle tehlike altına girebilir.', 'Balıkların yediği organizmalar tehlike altına girecek.', 'İnsanlar çok fazla balık yiyip hastalanabilirler.'], 'label': ['A', 'B', 'C', 'D']}",B,Balıkları yiyen organizmalar açlık nedeniyle tehlike altına girebilir.
610,Mercury_SC_416515,Hangi su kütlesinde en az bitki ve hayvan yaşıyor?\nŞıklar:\nA. okyanus\nB. aktarım\nC. buzul\nD. haliç,"{'text': ['okyanus', 'aktarım', 'buzul', 'haliç'], 'label': ['A', 'B', 'C', 'D']}",C,buzul
690,NYSEDREGENTS_2007_8_23,Doğrudan güneş ışığına maruz bırakıldığında hangi nesne en görünür ışık enerjisini emer?\nŞıklar:\n1. bir parça şeffaf cam\n2. bir kartopu\n3. parlak bir ayna\n4. siyah bir kazak,"{'text': ['bir parça şeffaf cam', 'bir kartopu', 'parlak bir ayna', 'siyah bir kazak'], 'label': ['1', '2', '3', '4']}",4,siyah bir kazak
2554,Mercury_SC_400700,Hangi gerçek neden Dünya'da hayat olduğunu ama Ay'da olmadığını en iyi açıklıyor?\nŞıklar:\nA. Ay'ın yerçekimi yoktur.\nB. Ay Güneş'in etrafında dönmez.\nC. Ay'da çok az su ve oksijen vardır.\nD. Ay Dünya'dan çok daha küçüktür.,"{'text': ['Ay'ın yerçekimi yoktur.', 'Ay Güneş'in etrafında dönmez.', 'Ay'da çok az su ve oksijen vardır.', 'Ay Dünya'dan çok daha küçüktür.'], 'label': ['A', 'B', 'C', 'D']}",C,Ay'da çok az su ve oksijen vardır.
580,Mercury_7215740,Bilim insanları bir enfeksiyonun nedenini belirlemek için bir hayvanın hücrelerini analiz etti. Bi

#### 400 Sampling

In [29]:
sample_dataset_name = "arc_400_sample.csv"
df_data_arc_400_sample = df_data_arc.sample(n=400, random_state=1911)
df_data_arc_400_sample.index
df_data_arc_400_sample.to_csv(os.path.join(SAMPLE_DATA_PATH, "arc", sample_dataset_name), index=False, encoding='utf-8')

Index([1945,  196, 1680,  667,  322, 1662, 1569,  794, 2021,  282,
       ...
       1285,  974,  149, 1747, 1857,  366,  121, 1610,  368,  250],
      dtype='int64', length=400)

In [30]:
df_data_arc_400_sample

,id,question,choices,answerKey,answer
1945,NYSEDREGENTS_2008_8_9,"Günümüzde pek çok araba, geçmişte üretilenlere göre daha iyi yakıt tüketimi sağlayacak şekilde tasarlanmıştır. Bu değişiklik bir ihtiyaçtan kaynaklanmıştır.\nŞıklar:\n1. geri dönüşüm malzemeleri\n2. güvenliği artırmak\n3. kimyasallar üretmek\n4. kaynakları koru","{'text': ['geri dönüşüm malzemeleri', 'güvenliği artırmak', 'kimyasallar üretmek', 'kaynakları koru'], 'label': ['1', '2', '3', '4']}",4,kaynakları koru
196,Mercury_SC_407674,Hangi diyagram enerjinin basit bir besin zincirinde nasıl aktığını en iyi şekilde gösterir?\nŞıklar:\nA. Yapraklar -> Tırtıl -> Kuş\nB. Ağaç -> Kuş -> Tırtıl\nC. Yapraklar -> Ağaç -> Tırtıl\nD. Tırtıl -> Yapraklar -> Kuş,"{'text': ['Yapraklar -> Tırtıl -> Kuş', 'Ağaç -> Kuş -> Tırtıl', 'Yapraklar -> Ağaç -> Tırtıl', 'Tırtıl -> Yapraklar -> Kuş'], 'label': ['A', 'B', 'C', 'D']}",A,Yapraklar -> Tırtıl -> Kuş
1680,LEAP__5_10315,Suyun sıcaklığı 10° Celsius'tan -10° Celsius'a değiştiğinde ne olur?\nŞıklar:\nA. Su katı halden gaz haline dönüşür.\nB. Su sıvı halden gaz haline dönüşür.\nC. Su sıvı halden katı hale dönüşür.\nD. Su katı halden sıvı hale dönüşür.,"{'text': ['Su katı halden gaz haline dönüşür.', 'Su sıvı halden gaz haline dönüşür.', 'Su sıvı halden katı hale dönüşür.', 'Su katı halden sıvı hale dönüşür.'], 'label': ['A', 'B', 'C', 'D']}",C,Su sıvı halden katı hale dönüşür.
667,Mercury_7233748,"Suyun bir kısmı tüketime yöneliktir. Su, kaynağına dönmek yerine rezervuarlardan buharlaşıyor veya mahsullerden çıkıyor. Bu tür bir kullanım hidrosferi büyük olasılıkla nasıl etkiler?\nŞıklar:\nA. Dünyadaki toplam su miktarını azaltarak\nB. Suyun Dünya sistemindeki yollarını değiştirerek\nC. Suyu su döngüsünde farklı aşamalara kaydırarak\nD. su döngüsünü değiştirecek şekilde iklimi etkileyerek","{'text': ['Dünyadaki toplam su miktarını azaltarak', 'Suyun Dünya sistemindeki yollarını değiştirerek', 'Suyu su döngüsünde farklı aşamalara kaydırarak', 'su döngüsünü değiştirecek şekilde iklimi etkileyerek'], 'label': ['A', 'B', 'C', 'D']}",C,Suyu su döngüsünde farklı aşamalara kaydırarak
322,WASL_2005_8_11,Aşağıdakilerden hangisi insanın kazanılmış bir özelliğidir?\nŞıklar:\nA. Göz rengi\nB. Saç rengi\nC. Yükseklik\nD. Sözlü vurgu,"{'text': ['Göz rengi', 'Saç rengi', 'Yükseklik', 'Sözlü vurgu'], 'label': ['A', 'B', 'C', 'D']}",D,Sözlü vurgu
...,...,...,...,...,...
366,MCAS_2006_9_27,Görünür ışık camdan geçer. Diğer elektromanyetik radyasyon türleri diğer malzemelerden benzer şekilde geçebilir. Aşağıdakilerden hangisi insan vücudunun bazı yerlerinden geçebildikleri için tıp teknolojisinde kullanılmaktadır?\nŞıklar:\nA. röntgen\nB. kızılötesi dalgalar\nC. mikrodalgalar\nD. ultraviyole ışınlar,"{'text': ['röntgen', 'kızılötesi dalgalar', 'mikrodalgalar', 'ultraviyole ışınlar'], 'label': ['A', 'B', 'C', 'D']}",A,röntgen
121,Mercury_SC_416518,Buzulları eriyen bir dağın şelaleli bir deresi vardır. Şelale dağın eteğindeki bir gölü dolduruyor. Su en soğuk nerede?\nŞıklar:\nA. buzul\nB. aktarım\nC. şelale\nD. göl,"{'text': ['buzul', 'aktarım', 'şelale', 'göl'], 'label': ['A', 'B', 'C', 'D']}",A,buzul
1610,Mercury_SC_400701,Aşağıdakilerden hangisi güneş gözlüklerinin gözleri korumaya yardımcı olduğu yollardan biridir?\nŞıklar:\nA. ışık ışınlarını gözlerden uzağa yansıtarak\nB. gözlere daha fazla ışık ışınını kırarak\nC. gözlere daha fazla ışık ışınını yansıtır\nD. ışık ışınlarını gözden uzağa doğru kırarak,"{'text': ['ışık ışınlarını gözlerden uzağa yansıtarak', 'gözlere daha fazla ışık ışınını kırarak', 'gözlere daha fazla ışık ışınını yansıtır', 'ışık ışınlarını gözden uzağa doğru kırarak'], 'label': ['A', 'B', 'C', 'D']}",A,ışık ışınlarını gözlerden uzağa yansıtarak
368,TIMSS_2011_4_pg64,Bunlardan hangisi karışımdır?\nŞıklar:\nA. tuzlu su\nB. şeker\nC. su buharı\nD. tuz,"{'text': ['tuzlu su', 'şeker', 'su buharı', 'tuz'], 'label': ['A', 'B', 'C', 'D']}",A,tuzlu su


#### Ensemble Method-1: 10 Örnekten Tekrarlı 10’lu Seçimler (Replacement=True)

In [31]:
for i in range(1, 11):
    print(f"Ensemble Method-1 Size:10_{i}")
    ens_1_dataset_name = f"arc_10_sample_ens_1_{i}.csv"
    df_data_ens_1_sample = df_data_arc_10_sample.sample(n=10, replace=True, random_state=i+5)
    df_data_ens_1_sample.to_csv(os.path.join(SAMPLE_DATA_PATH, "arc", ens_1_dataset_name), index=False, encoding='utf-8') # , compression='gzip'
    print(df_data_ens_1_sample.index)
    print()

Ensemble Method-1 Size:10_1
Index([580, 1476, 1063, 130, 580, 1312, 580, 1312, 1063, 1312], dtype='int64')

Ensemble Method-1 Size:10_2
Index([1063, 580, 610, 1476, 1476, 690, 690, 580, 690, 2554], dtype='int64')

Ensemble Method-1 Size:10_3
Index([1476, 1063, 1312, 580, 743, 2554, 1476, 2554, 130, 743], dtype='int64')

Ensemble Method-1 Size:10_4
Index([743, 610, 2554, 610, 1312, 610, 1063, 2554, 1312, 2554], dtype='int64')

Ensemble Method-1 Size:10_5
Index([580, 1063, 130, 1312, 580, 130, 1312, 2554, 580, 130], dtype='int64')

Ensemble Method-1 Size:10_6
Index([580, 130, 1312, 690, 1312, 690, 447, 2554, 130, 130], dtype='int64')

Ensemble Method-1 Size:10_7
Index([610, 1312, 447, 1476, 1476, 130, 610, 1312, 1063, 743], dtype='int64')

Ensemble Method-1 Size:10_8
Index([447, 130, 130, 610, 447, 1063, 580, 1476, 1063, 447], dtype='int64')

Ensemble Method-1 Size:10_9
Index([2554, 610, 690, 580, 610, 130, 2554, 580, 690, 610], dtype='int64')

Ensemble Method-1 Size:10_10
Index([2554, 7

#### Ensemble Method-2: 10 Örnekten 5’li Seçimler (Replacement=False)

In [32]:
for i in range(1, 11):
    print(f"Ensemble Method-2 Size:10_{i}")
    ens_2_dataset_name = f"arc_10_sample_ens_2_{i}.csv"
    df_data_ens_2_sample = df_data_arc_10_sample.sample(n=5, replace=False, random_state=i+20)
    df_data_ens_2_sample.to_csv(os.path.join(SAMPLE_DATA_PATH, "arc", ens_2_dataset_name), index=False, encoding='utf-8') # , compression='gzip'
    print(df_data_ens_2_sample.index)
    print()

Ensemble Method-2 Size:10_1
Index([1312, 447, 690, 1476, 743], dtype='int64')

Ensemble Method-2 Size:10_2
Index([447, 1312, 580, 610, 1476], dtype='int64')

Ensemble Method-2 Size:10_3
Index([743, 2554, 447, 580, 1063], dtype='int64')

Ensemble Method-2 Size:10_4
Index([580, 1063, 2554, 690, 743], dtype='int64')

Ensemble Method-2 Size:10_5
Index([743, 1312, 1476, 2554, 130], dtype='int64')

Ensemble Method-2 Size:10_6
Index([2554, 1063, 580, 690, 447], dtype='int64')

Ensemble Method-2 Size:10_7
Index([1063, 447, 580, 1312, 743], dtype='int64')

Ensemble Method-2 Size:10_8
Index([580, 447, 2554, 690, 1476], dtype='int64')

Ensemble Method-2 Size:10_9
Index([2554, 610, 1312, 690, 130], dtype='int64')

Ensemble Method-2 Size:10_10
Index([130, 1312, 447, 1476, 610], dtype='int64')



#### Ensemble Method-3: 10 Örnekten Tekrarsız 10’lu Seçimler (Replacement=False)

In [33]:
for i in range(1, 11):
    print(f"Ensemble Method-3 Size:10_{i}")
    ens_3_dataset_name = f"arc_10_sample_ens_3_{i}.csv"
    df_data_ens_3_sample = df_data_arc_10_sample.sample(n=10, replace=False, random_state=i+35)
    df_data_ens_3_sample.to_csv(os.path.join(SAMPLE_DATA_PATH, "arc", ens_3_dataset_name), index=False, encoding='utf-8') # , compression='gzip'
    print(df_data_ens_3_sample.index)
    print()

Ensemble Method-3 Size:10_1
Index([1476, 580, 690, 1063, 2554, 1312, 130, 610, 447, 743], dtype='int64')

Ensemble Method-3 Size:10_2
Index([1063, 580, 130, 1312, 690, 2554, 1476, 447, 610, 743], dtype='int64')

Ensemble Method-3 Size:10_3
Index([2554, 130, 690, 580, 447, 610, 1063, 1476, 743, 1312], dtype='int64')

Ensemble Method-3 Size:10_4
Index([447, 1312, 690, 2554, 743, 1476, 610, 1063, 130, 580], dtype='int64')

Ensemble Method-3 Size:10_5
Index([1063, 1476, 2554, 1312, 447, 580, 130, 743, 690, 610], dtype='int64')

Ensemble Method-3 Size:10_6
Index([743, 610, 690, 2554, 1312, 580, 447, 1063, 1476, 130], dtype='int64')

Ensemble Method-3 Size:10_7
Index([2554, 1312, 743, 130, 690, 447, 580, 1063, 1476, 610], dtype='int64')

Ensemble Method-3 Size:10_8
Index([1476, 580, 610, 2554, 447, 743, 1312, 690, 130, 1063], dtype='int64')

Ensemble Method-3 Size:10_9
Index([447, 690, 580, 130, 610, 2554, 743, 1312, 1476, 1063], dtype='int64')

Ensemble Method-3 Size:10_10
Index([447, 1063,

### Hellaswag Dataset

#### 10 Shot Sampling

In [35]:
sample_dataset_name = "hellaswag_10_sample.csv"
df_data_hellaswag_10_sample = df_data_hellaswag.sample(n=10, random_state=34)
df_data_hellaswag_10_sample.index
df_data_hellaswag_10_sample.to_csv(os.path.join(SAMPLE_DATA_PATH, "hellaswag", sample_dataset_name), index=False, encoding='utf-8')

Index([13686, 27298, 40857, 3638, 11623, 29526, 7932, 7800, 34330, 47315], dtype='int64')

In [36]:
df_data_hellaswag_10_sample.shape

(10, 13)

In [37]:
df_data_hellaswag_10_sample

,ind,activity_label,ctx_a,ctx_b,ctx,endings,source_id,split,split_type,label,question,answer,answerKey
13686,46795,Gitar çalıyorum,Bir adam oturmuş sigara içiyor. Beyaz bir gitar tutuyor ve çalıyor.,o,Bir adam oturmuş sigara içiyor. Beyaz bir gitar tutuyor ve çalıyor. O,"[ağzından dumanı üflemeye devam ediyor., sigarayı cebine atar ve gitar çalmaya devam eder., duraklar ve başını eğerek oynamaya devam eder., mikrofona şarkı söylemeye başlar.]",activitynet~v_a8dUtKcAunw,train,indomain,3,Bir adam oturmuş sigara içiyor. Beyaz bir gitar tutuyor ve çalıyor. O\nŞıklar:\nA- ağzından dumanı üflemeye devam ediyor.\nB- sigarayı cebine atar ve gitar çalmaya devam eder.\nC- duraklar ve başını eğerek oynamaya devam eder.\nD- mikrofona şarkı söylemeye başlar.,mikrofona şarkı söylemeye başlar.,D
27298,25405,İlişkiler,[header] Bir ilişkideki duygusal dağınıklığı nasıl temizleyebilirsiniz? [title] Diğer kişiyle ilgili hislerinize uyum sağlayın. [adım] Kendinize ilişkide sizi gerçekten neyin rahatsız ettiğini sorun. Rahatsız edici veya yüzleşmesi zor olsa bile duygularınızı kabul edin.,,[header] Bir ilişkideki duygusal dağınıklığı nasıl temizleyebilirsiniz? [title] Diğer kişiyle ilgili hislerinize uyum sağlayın. [adım] Kendinize ilişkide sizi gerçekten neyin rahatsız ettiğini sorun. Rahatsız edici veya yüzleşmesi zor olsa bile duygularınızı kabul edin.,"[Kendinize yalnız olduğunuzu ve rahatsız olmanızı gerektirecek hiçbir şey olmadığını hatırlatın. [alt adımlar] Bir ilişkiye duygusal olarak dahil olmanın anahtarı, başka bir kişiden kırgınlık veya incinme duygularını kalıcı olarak ortadan kaldırmanın imkansız olduğunu kabul etmektir., İçinizden hangi duyguların geçtiğine dikkat edin. Bazıları ile baş etmek diğerlerinden daha zordur, ancak bazıları değildir., [alt adımlar] Olumsuz duygularınızı bastırmaktan veya bunların kendiliğinden geçeceğini ummaktan kaçının. Fiziksel dağınıklık gibi, duygusal yük de siz ondan kurtulmak için harekete geçmediğiniz sürece kaybolmaz., Tavsiyede bulunmadan veya teşvik etmeden kendi duygularınızı diğer kişininkiyle karşılaştırın. [title] Diğer kişiye değil, her iki tarafın eylemlerine bakın.]",wikihow~41873,train,indomain,2,"[header] Bir ilişkideki duygusal dağınıklığı nasıl temizleyebilirsiniz? [title] Diğer kişiyle ilgili hislerinize uyum sağlayın. [adım] Kendinize ilişkide sizi gerçekten neyin rahatsız ettiğini sorun. Rahatsız edici veya yüzleşmesi zor olsa bile duygularınızı kabul edin.\nŞıklar:\nA- Kendinize yalnız olduğunuzu ve rahatsız olmanızı gerektirecek hiçbir şey olmadığını hatırlatın. [alt adımlar] Bir ilişkiye duygusal olarak dahil olmanın anahtarı, başka bir kişiden kırgınlık veya incinme duygularını kalıcı olarak ortadan kaldırmanın imkansız olduğunu kabul etmektir.\nB- İçinizden hangi duyguların geçtiğine dikkat edin. Bazıları ile baş etmek diğerlerinden daha zordur, ancak bazıları değildir.\nC- [alt adımlar] Olumsuz duygularınızı bastırmaktan veya bunların kendiliğinden geçeceğini ummaktan kaçının. Fiziksel dağınıklık gibi, duygusal yük de siz ondan kurtulmak için harekete geçmediğiniz sürece kaybolmaz.\nD- Tavsiyede bulunmadan veya teşvik etmeden kendi duygularınızı diğer kişininkiyle karşılaştırın. [title] Diğer kişiye değil, her iki tarafın eylemlerine bakın.","[alt adımlar] Olumsuz duygularınızı bastırmaktan veya bunların kendiliğinden geçeceğini ummaktan kaçının. Fiziksel dağınıklık gibi, duygusal yük de siz ondan kurtulmak için harekete geçmediğiniz sürece kaybolmaz.",C
40857,13560,Baton döndürme,Spor salonunun zemininde bir cop döndürücü duruyor. Rutinine başlıyor.,o,Spor salonunun zemininde bir cop döndürücü duruyor. Rutinine başlıyor. o,"[sopayı havaya fırlatır., havada coplarla hareket ediyor., kollarını biraz dışarı atıyor ve sonra asasını tekrar içeri sokuyor., yerde ileri geri gider.]",activitynet~v_e0-lO2jb8vo,val,indomain,0,Spor salonunun zemininde bir cop döndürücü duruyor. Rutinine başlıyor. o\nŞıklar:\nA- sopayı havaya fırlatır.\nB- havada coplarla hareket ediyor.\nC- kollarını 

#### 400 Sampling

In [38]:
sample_dataset_name = "hellaswag_400_sample.csv"
df_data_hellaswag_400_sample = df_data_hellaswag.sample(n=400, random_state=1911)
df_data_hellaswag_400_sample.index
df_data_hellaswag_400_sample.to_csv(os.path.join(SAMPLE_DATA_PATH, "hellaswag", sample_dataset_name), index=False, encoding='utf-8')

Index([17581, 16789, 44549, 41029,  2136, 47006, 18375,  5956,  6746,  6261,
       ...
       30592, 11354, 17714, 43402, 25283, 45144,  4243, 16895, 13467, 29150],
      dtype='int64', length=400)

In [39]:
df_data_hellaswag_400_sample

,ind,activity_label,ctx_a,ctx_b,ctx,endings,source_id,split,split_type,label,question,answer,answerKey
17581,5837,İş Dünyası,"[başlık] Bir moda tasarımı portföyü nasıl hazırlanır [başlık] Nelerin dahil edileceğine ilişkin talimatlar veya rehberlik için kontrol edin. [adım] Bir tasarım programına başvuruyorsanız, portföyünüzde ne görmek istediklerine dair iyi bir fikir vermelidirler. Profesyonel bir portföy tasarlıyorsanız açık talimatlarınız olmayabilir.",,"[başlık] Bir moda tasarımı portföyü nasıl hazırlanır [başlık] Nelerin dahil edileceğine ilişkin talimatlar veya rehberlik için kontrol edin. [adım] Bir tasarım programına başvuruyorsanız, portföyünüzde ne görmek istediklerine dair iyi bir fikir vermelidirler. Profesyonel bir portföy tasarlıyorsanız açık talimatlarınız olmayabilir.","[Ancak, bilgiyi bulmanın ve onu her zamankinden daha iyi derinlemesine açıklamanın gerektiğini unutmayın. Burada size rehberlik etmek için kullanabileceğiniz bazı yararlı rehberlik broşürleri bulunmaktadır., Genel olarak yaratıcı araştırmayı, 3 boyutluyu 2 boyutluya çevirme yeteneğinizi gösteren çizimleri, renk çalışmalarını ve varsa yaptığınız 3 boyutlu çalışmanın fotoğraflarını eklemelisiniz. [alt adımlar] Portföyünüzü, yapmak istediğiniz belirli uygulamaya veya iş türüne göre uyarlayın., Ne ekleyeceğinizden emin değilseniz sorun. Farklı insanlardan öğrenecek deneyiminiz var mı yoksa doğru iş türünü bulmak için profesyonel deneyime mi ihtiyacınız var? [alt adımlar] Mükemmel, uygulanabilir ürünleriniz olduğundan emin olun., İstediğiniz çözüm noktasına ulaşmak için birine tüyo vermeniz gerekebilir. [alt adımlar] Profesyonel bir portföy tasarlıyor olsanız bile yine de profesyonel olmanız gerektiğini unutmayın.]",wikihow~9678,train,indomain,1,"[başlık] Bir moda tasarımı portföyü nasıl hazırlanır [başlık] Nelerin dahil edileceğine ilişkin talimatlar veya rehberlik için kontrol edin. [adım] Bir tasarım programına başvuruyorsanız, portföyünüzde ne görmek istediklerine dair iyi bir fikir vermelidirler. Profesyonel bir portföy tasarlıyorsanız açık talimatlarınız olmayabilir.\nŞıklar:\nA- Ancak, bilgiyi bulmanın ve onu her zamankinden daha iyi derinlemesine açıklamanın gerektiğini unutmayın. Burada size rehberlik etmek için kullanabileceğiniz bazı yararlı rehberlik broşürleri bulunmaktadır.\nB- Genel olarak yaratıcı araştırmayı, 3 boyutluyu 2 boyutluya çevirme yeteneğinizi gösteren çizimleri, renk çalışmalarını ve varsa yaptığınız 3 boyutlu çalışmanın fotoğraflarını eklemelisiniz. [alt adımlar] Portföyünüzü, yapmak istediğiniz belirli uygulamaya veya iş türüne göre uyarlayın.\nC- Ne ekleyeceğinizden emin değilseniz sorun. Farklı insanlardan öğrenecek deneyiminiz var mı yoksa doğru iş türünü bulmak için profesyonel deneyime mi ihtiyacınız var? [alt adımlar] Mükemmel, uygulanabilir ürünleriniz olduğundan emin olun.\nD- İstediğiniz çözüm noktasına ulaşmak için birine tüyo vermeniz gerekebilir. [alt adımlar] Profesyonel bir portföy tasarlıyor olsanız bile yine de profesyonel olmanız gerektiğini unutmayın.","Genel olarak yaratıcı araştırmayı, 3 boyutluyu 2 boyutluya çevirme yeteneğinizi gösteren çizimleri, renk çalışmalarını ve varsa yaptığınız 3 boyutlu çalışmanın fotoğraflarını eklemelisiniz. [alt adımlar] Portföyünüzü, yapmak istediğiniz belirli uygulamaya veya iş türüne göre uyarlayın.",B
16789,4239,İş Dünyası,"[header] Okulda nasıl favori öğretmen olunur? [title] Mantıklı olun. [adım] Öğrenciler kararlı ama adil olan öğretmenlere yanıt verirler. Küçük ihlallerden dolayı çocukları çok fazla azarlamayın; makul olun ve onları sorumlu tutun, ancak aşırıya kaçmayın.",,"[header] Okulda nasıl favori öğretmen olunur? [title] Mantıklı olun. [adım] Öğrenciler kararlı ama adil olan öğretmenlere yanıt verirler. Küçük ihlallerden dolayı çocukları çok fazla azarlamayın; makul olun ve onları sorumlu tutun, ancak aşırıya kaçmayın.","[Onları sık sık cezalandırsanız bile yine de iyi bir öğretmen olabilirsiniz. [alt adımlar] İlk gününde kaba davranan çocukları eleştirmeyi

#### Ensemble Method-1: 10 Örnekten Tekrarlı 10’lu Seçimler (Replacement=True)

In [40]:
for i in range(1, 11):
    print(f"Ensemble Method-1 Size:10_{i}")
    ens_1_dataset_name = f"hellaswag_10_sample_ens_1_{i}.csv"
    df_data_ens_1_sample = df_data_hellaswag_10_sample.sample(n=10, replace=True, random_state=i+5)
    # df_data_ens_1_sample.to_csv(os.path.join(SAMPLE_DATA_PATH, "hellaswag", ens_1_dataset_name), index=False, encoding='utf-8') # , compression='gzip'
    print(df_data_ens_1_sample.index)
    print()

Ensemble Method-1 Size:10_1
Index([47315, 3638, 11623, 13686, 47315, 27298, 47315, 27298, 11623, 27298], dtype='int64')

Ensemble Method-1 Size:10_2
Index([11623, 47315, 7932, 3638, 3638, 7800, 7800, 47315, 7800, 34330], dtype='int64')

Ensemble Method-1 Size:10_3
Index([3638, 11623, 27298, 47315, 29526, 34330, 3638, 34330, 13686, 29526], dtype='int64')

Ensemble Method-1 Size:10_4
Index([29526, 7932, 34330, 7932, 27298, 7932, 11623, 34330, 27298, 34330], dtype='int64')

Ensemble Method-1 Size:10_5
Index([47315, 11623, 13686, 27298, 47315, 13686, 27298, 34330, 47315, 13686], dtype='int64')

Ensemble Method-1 Size:10_6
Index([47315, 13686, 27298, 7800, 27298, 7800, 40857, 34330, 13686, 13686], dtype='int64')

Ensemble Method-1 Size:10_7
Index([7932, 27298, 40857, 3638, 3638, 13686, 7932, 27298, 11623, 29526], dtype='int64')

Ensemble Method-1 Size:10_8
Index([40857, 13686, 13686, 7932, 40857, 11623, 47315, 3638, 11623, 40857], dtype='int64')

Ensemble Method-1 Size:10_9
Index([34330, 79

#### Ensemble Method-2: 10 Örnekten 5’li Seçimler (Replacement=False)

In [41]:
for i in range(1, 11):
    print(f"Ensemble Method-2 Size:10_{i}")
    ens_2_dataset_name = f"hellaswag_10_sample_ens_2_{i}.csv"
    df_data_ens_2_sample = df_data_hellaswag_10_sample.sample(n=5, replace=False, random_state=i+20)
    # df_data_ens_2_sample.to_csv(os.path.join(SAMPLE_DATA_PATH, "hellaswag", ens_2_dataset_name), index=False, encoding='utf-8') # , compression='gzip'
    print(df_data_ens_2_sample.index)
    print()

Ensemble Method-2 Size:10_1
Index([27298, 40857, 7800, 3638, 29526], dtype='int64')

Ensemble Method-2 Size:10_2
Index([40857, 27298, 47315, 7932, 3638], dtype='int64')

Ensemble Method-2 Size:10_3
Index([29526, 34330, 40857, 47315, 11623], dtype='int64')

Ensemble Method-2 Size:10_4
Index([47315, 11623, 34330, 7800, 29526], dtype='int64')

Ensemble Method-2 Size:10_5
Index([29526, 27298, 3638, 34330, 13686], dtype='int64')

Ensemble Method-2 Size:10_6
Index([34330, 11623, 47315, 7800, 40857], dtype='int64')

Ensemble Method-2 Size:10_7
Index([11623, 40857, 47315, 27298, 29526], dtype='int64')

Ensemble Method-2 Size:10_8
Index([47315, 40857, 34330, 7800, 3638], dtype='int64')

Ensemble Method-2 Size:10_9
Index([34330, 7932, 27298, 7800, 13686], dtype='int64')

Ensemble Method-2 Size:10_10
Index([13686, 27298, 40857, 3638, 7932], dtype='int64')



#### Ensemble Method-3: 10 Örnekten Tekrarsız 10’lu Seçimler (Replacement=False)

In [42]:
for i in range(1, 11):
    print(f"Ensemble Method-3 Size:10_{i}")
    ens_3_dataset_name = f"hellaswag_10_sample_ens_3_{i}.csv"
    df_data_ens_3_sample = df_data_hellaswag_10_sample.sample(n=10, replace=False, random_state=i+35)
    # df_data_ens_3_sample.to_csv(os.path.join(SAMPLE_DATA_PATH, "hellaswag", ens_3_dataset_name), index=False, encoding='utf-8') # , compression='gzip'
    print(df_data_ens_3_sample.index)
    print()

Ensemble Method-3 Size:10_1
Index([3638, 47315, 7800, 11623, 34330, 27298, 13686, 7932, 40857, 29526], dtype='int64')

Ensemble Method-3 Size:10_2
Index([11623, 47315, 13686, 27298, 7800, 34330, 3638, 40857, 7932, 29526], dtype='int64')

Ensemble Method-3 Size:10_3
Index([34330, 13686, 7800, 47315, 40857, 7932, 11623, 3638, 29526, 27298], dtype='int64')

Ensemble Method-3 Size:10_4
Index([40857, 27298, 7800, 34330, 29526, 3638, 7932, 11623, 13686, 47315], dtype='int64')

Ensemble Method-3 Size:10_5
Index([11623, 3638, 34330, 27298, 40857, 47315, 13686, 29526, 7800, 7932], dtype='int64')

Ensemble Method-3 Size:10_6
Index([29526, 7932, 7800, 34330, 27298, 47315, 40857, 11623, 3638, 13686], dtype='int64')

Ensemble Method-3 Size:10_7
Index([34330, 27298, 29526, 13686, 7800, 40857, 47315, 11623, 3638, 7932], dtype='int64')

Ensemble Method-3 Size:10_8
Index([3638, 47315, 7932, 34330, 40857, 29526, 27298, 7800, 13686, 11623], dtype='int64')

Ensemble Method-3 Size:10_9
Index([40857, 7800, 

## Show Metadata, Shape, Statistics and Missing Table

In [ ]:
# df_ins_data_all.info()

In [ ]:
# df_ins_data_all.shape

In [ ]:
# missing_values(df=df_ins_data_all, threshold=0, asc_sorting=False)

In [ ]:
col_list = df_data_gsm.columns.to_list()

for col in col_list:
    df_data_gsm[f"{col}Length"] = df_data_gsm[col].apply(lambda x: len(str(x)) if pd.notnull(x) else 0)
    df_data_gsm[f"{col}WordCount"] = df_data_gsm[col].apply(lambda x: len(str(x).split()) if pd.notnull(x) else 0)

In [ ]:
# df_data_gsm.head()
# df_data_gsm.sample(3)

# df_data_gsm_sort = df_data_gsm.sort_values(by=['questionLength'], ascending=True)
# df_data_gsm_sort

In [ ]:
# df_data_gsm.columns

In [ ]:
df_data_gsm[['questionLength', 'questionWordCount', 'raw_answerLength', 'raw_answerWordCount', 'answerLength', 'answerWordCount', 'short_answerLength', 'short_answerWordCount', 'formated_answerLength', 'formated_answerWordCount']].describe(percentiles=[0.80, 0.85, 0.90, 0.95, 0.99]).T

In [ ]:
# df_data_gsm[['question', 'raw_answer', 'answer', 'short_answer', 'formated_answer']]

## Split Data

In [ ]:
df_ins_data_all['soru'] = df_ins_data_all['talimat'].fillna('') + ' ' + df_ins_data_all['giris'].fillna('')
df_ins_data_all[["talimat", "giris", "soru", "cevap"]].head()

In [ ]:
df_sample_1 = df_ins_data_all[df_ins_data_all['giris'].notna()].sample(n=1000, random_state=42)
df_sample_1_idx = df_sample_1.index
df_sample_1.head()

df_sample_2 = df_ins_data_all[~(df_ins_data_all['giris'].notna())].sample(n=1000, random_state=42)
df_sample_2_idx = df_sample_2.index
df_sample_2.head()

df_sample = pd.concat([df_sample_1, df_sample_2])
df_sample['soru'] = df_sample['talimat'].fillna('') + " " + df_sample['giris'].fillna('')

df_ins_data_all[col].apply(lambda x: len(str(x).split()) if pd.notnull(x) else 0)

df_sample.to_csv(os.path.join(RAW_DATA_PATH, "sampled_turkish_instructions.csv"), index=False)

df_sample = df_sample[["talimat", "giris", "soru", "cevap"]].reset_index(drop=True)
df_sample = pd.read_csv(os.path.join(RAW_DATA_PATH, "sampled_turkish_instructions.csv"))
df_sample.head()

In [ ]:
df_sample.sample(5)

In [ ]:
df_sample.shape